#### Perceptron de Camada Única

Do inglês Singles Layer Perceptron (SLP), é uma das primeiras e mais antigas redes neurais criada. Foi proposta por Frank Rosenblatt em 1958. Por ser o tipo mais imples de rede neural, ela apenas classifica casos linearmente separáveis de forma binária. 

Uma SLP é uma rede de alimentação direta baseada em uma função de transferência. Essas funções são equações matemáticas que determinam a saída de uma rede neural.

Componentes do Perceptron:
- Input: entradas reais ou binárias;
- Peso: cada entrada é associada à um peso, representando o seu grau de importância;
- Bias: esse termo permite o neurônio ser ativado até quando a entrada é zero, mitigando o problema de _vanishing gradient_;
- Função de ativação: o perceptron usa uma função degrau para determinar se a soma do produto da entrada pelo peso, somado ao bias, está acima ou abaixo de um certo limite.

#### Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np

#### Função de treinamento

In [ ]:
def treinamento(X_train, y_train, qtd_atributos, tx_aprendizagem, epocas, seed):
    """
        Argumentos:
        X_train -- amostras de treino
        y_train -- resultados desejados das amostras de treino
        qtd_atributos -- quantidade de features
        tx_aprendizagem -- taxa de aprendizado da rede
        epocas -- cada apresentação completa de todas as amostras pertencentes ao subconjunto de treinamento
        seed -- para garantir que sempre será utilizado o mesmo número aleatório
        
        Saída:
        parâmetros -- python dict:
                        peso aprendido -- pesos aprendidos ao final do treinamento
                        epocas executadas -- quantidade total de épocas ao final do treinamento
                        evolucao do erro -- lista contendo a evolução do erro ao longo das épocas
                        acuracia -- relação entre o total de acertos e o total de amostras
    """

    np.random.seed(seed)

    pesos = np.random.uniform(low=-0.5, high=0.5, size=((qtd_atributos + 1),))  # vetor de peso (atributos + bias)
    eta = tx_aprendizagem                                                       # taxa de aprendizagem
    epoca = 0                                                                   # contador de epoca
    evolucao_erro = []

    while epoca < epocas:
        erro = 0
        acertos = 0

        for i in range(len(X_train)):
            x = np.append(1, X_train[i])    # adiciona o bias na amostra X_train
            y_desejado = y_train[i]

            u = np.dot(pesos, x)                                # potencial de ativação (produto escalar)

            if u >= 0:                                          # função de ativação -> y_pred = sinal(u)
                y_pred = 1
            else:
                y_pred = -1

            if y_pred != y_desejado:                            # regra de aprendizado de Hebb
                peso = peso + eta * (y_desejado - y_pred) * x
                erro += 1
            else:
                acertos += 1

            evolucao_erro.append(erro/len(X_train))     # evolução do erro ao longo das épocas

        epoca += 1

        if erro == 0:
            break
    
    return {
        "peso aprendido": pesos,
        "epocas executadas": epoca + 1,
        "evolucao do erro": evolucao_erro,
        "acuracia": acertos/len(X_train)
    }
                

#### Função de teste

In [ ]:
def teste(X_test, y_test, peso):
    """
        Argumentos:
        X_test -- amostras de teste
        y_test -- resultados desejados das amostras de teste
        peso -- pesos aprendidos durante o treinamento
        
        Saída:
        parâmetros -- python dict:
                        acuracia -- relação entre o total de acertos e o total de amostras
    """

    acertos = 0                          # contador de acertos
    
    for i in range(len(X_test)):
        x = np.append(1, X_test[i])     # adiciona o bias na amostra X_test
        y_desejado = y_test[i]

        u = np.dot(x, peso)             # potencial de ativação com base no peso aprendido

        if u >= 0:                      # função de ativação -> y_pred = sinal(u)
            y_pred = 1
        else:
            y_pred = -1

        if y_pred == y_desejado:
            acertos += 1

    return {
        "acuracia": acertos/len(X_test)
    }

#### Importando os datasets de treinamento e teste

In [80]:
# Carregando os datasets
df_train_loaded = pd.read_csv("arquivos_csv/train_dataset1.csv")
df_test_loaded = pd.read_csv("arquivos_csv/test_dataset1.csv")


# Separando os dados de treinamento
X_train = df_train_loaded.drop("label", axis=1).values.T    # (n_features, n_amostras)
y_train = df_train_loaded["label"].values.reshape(1, -1)    # (1, n_amostras)

# Separando os dados de teste
X_test = df_test_loaded.drop("label", axis=1).values.T
y_test = df_test_loaded["label"].values.reshape(1, -1)

#### Referências

- Slides da aula;
- https://pyimagesearch.com/2021/05/06/implementing-the-perceptron-neural-network-with-python/